In [ ]:
import os
from copy import deepcopy

import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats
import scipy.ndimage
from tqdm.notebook import tqdm


import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import cartopy.crs as ccrs
import matplotlib.dates as mdates
import matplotlib.lines as mpll

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
plt.rcParams['text.usetex'] = False
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
std_corr = np.sqrt(41 / 40)
DENSITY = 1000

In [ ]:
cluster = distributed.LocalCluster(n_workers=40, threads_per_worker=1, memory_limit='2GB')
client = distributed.Client(cluster)
client

# Load data

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/da_enkf_for_soil'

## H2O

In [ ]:
vr_h2o_path = os.path.join(base_path, '016', 'h2o_cleaned.nc')
vr_h2o = xr.open_dataset(vr_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_h2o_first_path = os.path.join(base_path, '015', 'h2o_cleaned.nc')
da_h2o_first = xr.open_dataset(da_h2o_first_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_h2o_path = os.path.join(base_path, '020', 'h2o_cleaned.nc')
da_h2o = xr.open_dataset(da_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((40, 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_h2o = xr.concat([da_h2o_first.sel(time='2015-07-31 12:00'), da_h2o], dim='time')

In [ ]:
da_ana_path = os.path.join(base_path, '020', 'da_offline_1d_enkf_nature.nc')
da_ana = xr.open_dataarray(da_ana_path).squeeze(drop=True).chunk(( 1, 302, 267))[..., 30:-30, 30:-30]

In [ ]:
da_h2o_bg = da_h2o.sel(time=da_h2o.indexes['time'].minute==0)
da_h2o_ana = da_ana
da_h2o_ana['time'] = da_h2o_bg.indexes['time']

In [ ]:
const_path = '/work/um0203/u300636/for2131/runs/utilities/clm_aux.nc'
const_data = xr.open_dataset(const_path).isel(time=0)
sat_point = const_data['WATSAT'].isel(levsoi=4)[..., 30:-30, 30:-30]

## Get ensemble covariance

In [ ]:
err_bg = (da_h2o_bg.mean('ensemble')-vr_h2o)**2
err_ana = (da_h2o_ana-vr_h2o)**2
impact = np.sqrt(err_ana.mean(['lat', 'lon']))-np.sqrt(err_bg.mean(['lat', 'lon']))

In [ ]:
bg_smi = da_h2o_bg / sat_point

In [ ]:
da_dry = bg_smi.mean('ensemble') < 0.2
da_wet = bg_smi.mean('ensemble') > 0.5
da_mixed = ~np.logical_or(da_dry, da_wet)

In [ ]:
impact_dry = np.sqrt(err_ana.where(da_dry).mean(['lat', 'lon'])) - np.sqrt(err_bg.where(da_dry).mean(['lat', 'lon']))
impact_wet = np.sqrt(err_ana.where(da_wet).mean(['lat', 'lon'])) - np.sqrt(err_bg.where(da_wet).mean(['lat', 'lon']))
impact_mixed = np.sqrt(err_ana.where(da_mixed).mean(['lat', 'lon'])) - np.sqrt(err_bg.where(da_mixed).mean(['lat', 'lon']))

In [ ]:
impact_cycle = impact[7:].groupby('time.hour').mean(['time']).roll(hour=1, roll_coords=True).load()
dry_cycle = impact_dry[7:].groupby('time.hour').mean(['time']).roll(hour=1, roll_coords=True).load()
wet_cycle = impact_wet[7:].groupby('time.hour').mean(['time']).roll(hour=1, roll_coords=True).load()
mixed_cycle = impact_mixed[7:].groupby('time.hour').mean(['time']).roll(hour=1, roll_coords=True).load()

In [ ]:
figsize = [s*0.48 for s in plt.rcParams['figure.figsize']]
fig, ax = plt.subplots(figsize=figsize, sharex=True)
ax.plot(np.arange(-1, 23, 1), wet_cycle, lw=0.8, c='C0', label='moist', ls='--')
ax.plot(np.arange(-1, 23, 1), mixed_cycle, lw=0.8, c='salmon', label='mixed', ls='-.')
ax.plot(np.arange(-1, 23, 1), dry_cycle, lw=0.8, c='saddlebrown', label='dry', ls='dotted')
ax.plot(np.arange(-1, 23, 1), impact_cycle, lw=0.8, c='black', label='total')
legend = ax.legend(loc=3)
legend.get_frame().set_linewidth(0.8)
ax.axhline(c='black', lw=0.5)
ax.set_ylabel('Potential impact\n(m$^3$/m$^3$)')
ax.set_xticks(np.arange(0, 24, 3))
ax.set_xlabel('Time of day (UTC)')
ax.set_ylim(-3.3E-3, 0.5E-3)

fig.savefig('../figures/fig_09_soil_moisture_impact.png', dpi=300)